# CellChat

In [ ]:
options(warn=-1, stringsAsFactors=FALSE)

In [ ]:
library_load <- suppressMessages(
    
    list(
        
        # CellTalk
        library(CellChat), 
        
        # Seurat 
        library(Seurat), 
        
        # Data 
        library(tidyverse), 
        library(reshape2), 
        
        # Plotting 
        library(ComplexHeatmap), 
        library(circlize), 
        library(viridis), 
        library(ggplotify), 
        library(ggrepel), 
        library(cowplot)
        
    )
)

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
ht_opt$message=FALSE # ComplexHeatmap 

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source files
source("plotting_global.R")
source("bin/cci_pp.R")
source("bin/cci_pl.R")

In [ ]:
# Plotting Theme
ggplot2::theme_set(theme_global_set(1)) # From project global source()

# Parameter and files 

In [ ]:
so_file <- "data/scRNAseq/object/pp.rds"

In [ ]:
cache_cco <- TRUE

# Import 

In [ ]:
so <- readRDS(so_file)

In [ ]:
# Subset by celltype 
so <- subset(so, subset=celltype_low %in% c("MEP", "Proerythroblast", "Erythroblast", "cMo", "intMo", "ncMo", "RPM", "cDC2", "Basophil"))

In [ ]:
# Filter genes
cnt <- GetAssayData(so, assay="RNA", slot="counts")
cnt <- cnt[rowSums(cnt>=3)>=3, ]

so <- CreateSeuratObject(counts=cnt, meta.data=so@meta.data)
    
# Get normalized counts 
so <- NormalizeData(so, normalization.method="LogNormalize", verbose=FALSE)

# Subset data base 

In [ ]:
db <- CellChat::CellChatDB.mouse
db <- CellChat::subsetDB(db, search=c("Cell-Cell Contact", "Secreted Signaling"), key="annotation")

# Add custom interaction 

In [ ]:
options(stringsAsFactors=FALSE)
interaction <- read.csv("data/reference/cellchat/interaction.csv", row.names=1)
interaction[is.na(interaction)] <- ""
db$interaction <- rbind(db$interaction, interaction)

# Get overall signaling genes

In [ ]:
# Get normalized data 
nrm <- GetAssayData(so, assay="RNA", slot="data")

# Get meta data 
meta <- so@meta.data
meta <- droplevels(meta)

# Create cellchat object
cco <- createCellChat(object=nrm, meta=meta, group.by="celltype_low")
cco <- setIdent(cco, ident.use="celltype_low")

# Use DB subset 
cco@DB <- db

# Subset for signaling genes
cco <- subsetData(cco, features=NULL)

# Get signaling gnes
genes_signaling <- rownames(cco@data.signaling)

## Pre-process cellchat objects for each sample group condition 

In [ ]:
cco_pp <- function(so, group_by) {
    
    # Get normalized data 
    nrm <- GetAssayData(so, assay="RNA", slot="data")
    
    # Get meta data 
    meta <- so@meta.data
    meta <- droplevels(meta)
    
    cco <- createCellChat(object=nrm, meta=meta, group.by=group_by)
    cco <- setIdent(cco, ident.use=group_by)
    
    cco@DB <- db # Add DB slot with interaction, complex, cofactor, and geneInfo data frames 
    cco <- subsetData(cco, features=genes_signaling) # Subset data for signaling genes computed above 
    # cco <- identifyOverExpressedGenes(cco, thresh.p=1) # Identify DE signaling genes stored in object@var.features. Disabled for now since I want to keep all genes for analysis.
    cco@var.features <- list(features=rownames(cco@data))
    cco <- identifyOverExpressedInteractions(cco, features.name="features") # Filter interaction based on over expressed genes. Adds @LR$LRsig which is a pruned version of cco@DB$interaction
    # cco <- smoothData(cco, adj=PPI.mouse) # Project protein data 
    cco <- computeCommunProb(cco, raw.use=TRUE, population.size=TRUE, type="truncatedMean", trim=0.01) # Using the information from DB compute interaction strength (probability) in object@net$prob and associated p-values in object@net$pval.
    cco <- computeCommunProbPathway(cco) # Summarise L-R interaction on pathway level. Return object@netP$prob is the pathway probability and significant pathways are stored in object@netP$pathways
    cco <- aggregateNet(cco, thresh=0.05) # Updates object@net with counts and weights 
    cco <- netAnalysis_computeCentrality(cco) 
    
    return(cco)
    
}

In [ ]:
if(!cache_cco) {
    
    cco_1 <- cco_pp(subset(so, subset=sample_group=="Bl6_NaCl_D6"), group_by="celltype_low")
    saveRDS(cco_1, "data/scRNAseq/cci/cco_bl6_nacl_d6.rds")

} else {
    
    cco_1 <- readRDS("data/scRNAseq/cci/cco_bl6_nacl_d6.rds")
    
}

In [ ]:
if(!cache_cco) {
    
    cco_2 <- cco_pp(subset(so, subset=sample_group=="Bl6_CpG_D6"), group_by="celltype_low")
    saveRDS(cco_2, "data/scRNAseq/cci/cco_bl6_cpg_d6.rds")

} else {
    
    cco_2 <- readRDS("data/scRNAseq/cci/cco_bl6_cpg_d6.rds")
    
}

In [ ]:
if(!cache_cco) {
    
    cco_3 <- cco_pp(subset(so, subset=sample_group=="IFNAR_fl_Baseline_D0"), group_by="celltype_low")
    saveRDS(cco_3, "data/scRNAseq/cci/cco_ifnar_fl_baseline_d0.rds")

} else {
    
    cco_3 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_baseline_d0.rds")
    
}

In [ ]:
if(!cache_cco) {
    
    cco_4 <- cco_pp(subset(so, subset=sample_group=="IFNAR_fl_CpG_D1"), group_by="celltype_low")
    saveRDS(cco_4, "data/scRNAseq/cci/cco_ifnar_fl_cpg_d1.rds")

} else {
    
    cco_4 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_cpg_d1.rds")
    
}

In [ ]:
if(!cache_cco) {
    
    cco_5 <- cco_pp(subset(so, subset=sample_group=="IFNAR_fl_CpG_D3"), group_by="celltype_low")
    saveRDS(cco_5, "data/scRNAseq/cci/cco_ifnar_fl_cpg_d3.rds")

} else {
    
    cco_5 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_cpg_d3.rds")
    
}

In [ ]:
if(!cache_cco) {
    
    cco_6 <- cco_pp(subset(so, subset=sample_group=="IFNAR_fl_LysM_cre_Baseline_D0"), group_by="celltype_low")
    saveRDS(cco_6, "data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_baseline_d0.rds")

} else {
    
    cco_6 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_baseline_d0.rds")
    
}

In [ ]:
if(!cache_cco) {
    
    cco_7 <- cco_pp(subset(so, subset=sample_group=="IFNAR_fl_LysM_cre_CpG_D1"), group_by="celltype_low")
    saveRDS(cco_7, "data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_cpg_d1.rds")

} else {
    
    cco_7 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_cpg_d1.rds")
    
}

In [ ]:
if(!cache_cco) {
    
    cco_8 <- cco_pp(subset(so, subset=sample_group=="IFNAR_fl_LysM_cre_CpG_D3"), group_by="celltype_low")
    saveRDS(cco_8, "data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_cpg_d3.rds")

} else {
    
    cco_8 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_cpg_d3.rds")
    
}

# Save full results 

In [ ]:
get_lr_res <- function(cco) {

    interaction <- db[["interaction"]] %>% 
        dplyr::select(interaction_name, pathway_name, ligand.symbol, receptor.symbol) %>% 
        dplyr::rename(ligand_symbol=ligand.symbol, receptor_symbol=receptor.symbol) %>% 
        dplyr::mutate(ligand_symbol=gsub(", ", "_", ligand_symbol), receptor_symbol=gsub(", ", "_", receptor_symbol))

    lr_res <- reshape2::melt(cco@net[["prob"]]) %>% 
        dplyr::rename(source=Var1, target=Var2, interaction_name=Var3, prob=value) %>% 
        dplyr::mutate(pval=reshape2::melt(cco@net[["pval"]])[[4]]) %>% 
        dplyr::left_join(., interaction, by=join_by(interaction_name))



    return(lr_res)
}

In [ ]:
lr_res_l <- list(
    
    get_lr_res(cco_1) %>% dplyr::mutate(sample_group="Bl6_NaCl_D6"),
    get_lr_res(cco_2) %>% dplyr::mutate(sample_group="Bl6_CpG_D6"),
    get_lr_res(cco_3) %>% dplyr::mutate(sample_group="IFNAR_fl_Baseline_D0"),
    get_lr_res(cco_4) %>% dplyr::mutate(sample_group="IFNAR_fl_CpG_D1"),
    get_lr_res(cco_5) %>% dplyr::mutate(sample_group="IFNAR_fl_CpG_D3"),
    get_lr_res(cco_6) %>% dplyr::mutate(sample_group="IFNAR_fl_LysM_cre_Baseline_D0"),
    get_lr_res(cco_7) %>% dplyr::mutate(sample_group="IFNAR_fl_LysM_cre_CpG_D1"),
    get_lr_res(cco_8) %>% dplyr::mutate(sample_group="IFNAR_fl_LysM_cre_CpG_D3")

)
lr_res <- do.call(rbind, lr_res_l)

In [ ]:
write.csv(lr_res, "result/cci/lr_res.csv")

# Merge CellChat data objects 

In [ ]:
cco_diff_1 <- list(Bl6_NaCl_D6=cco_1, Bl6_CpG_D6=cco_2)
cco_diff_1 <- mergeCellChat(cco_diff_1, add.names=names(cco_diff_1))
cco_diff_1@meta$sample_group <- factor(cco_diff_1@meta$sample_group, levels=c("Bl6_NaCl_D6", "Bl6_CpG_D6"))

In [ ]:
cco_diff_2 <- list(IFNAR_fl_Baseline_D0=cco_3, IFNAR_fl_CpG_D1=cco_4)
cco_diff_2 <- mergeCellChat(cco_diff_2, add.names=names(cco_diff_2))
cco_diff_2@meta$sample_group <- factor(cco_diff_2@meta$sample_group, levels=c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1"))

In [ ]:
cco_diff_3 <- list(IFNAR_fl_Baseline_D0=cco_3, IFNAR_fl_CpG_D3=cco_5)
cco_diff_3 <- mergeCellChat(cco_diff_3, add.names=names(cco_diff_3))
cco_diff_3@meta$sample_group <- factor(cco_diff_3@meta$sample_group, levels=c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D3"))

In [ ]:
cco_diff_4 <- list(IFNAR_fl_CpG_D1=cco_4, IFNAR_fl_LysM_cre_CpG_D1=cco_7)
cco_diff_4 <- mergeCellChat(cco_diff_4, add.names=names(cco_diff_4))
cco_diff_4@meta$sample_group <- factor(cco_diff_4@meta$sample_group, levels=c("IFNAR_fl_CpG_D1", "IFNAR_fl_LysM_cre_CpG_D1"))

In [ ]:
cco_diff_5 <- list(IFNAR_fl_CpG_D1=cco_4, IFNAR_fl_LysM_cre_CpG_D3=cco_8)
cco_diff_5 <- mergeCellChat(cco_diff_5, add.names=names(cco_diff_5))
cco_diff_5@meta$sample_group <- factor(cco_diff_5@meta$sample_group, levels=c("IFNAR_fl_CpG_D1", "IFNAR_fl_LysM_cre_CpG_D3"))

# Heatmap of number of interactions 

In [ ]:
options(repr.plot.width=40, repr.plot.height=10)

hm_1 <- hm_net(cellchat=cco_1, slot="count", title="Interaction count Bl6_NaCl_D6") %>% ggplotify::as.ggplot()
hm_2 <- hm_net(cellchat=cco_2, slot="count", title="Interaction count Bl6_CpG_D6") %>% ggplotify::as.ggplot()
hm_3 <- hm_net(cellchat=cco_3, slot="count", title="Interaction count IFNAR_fl_Baseline_D0") %>% ggplotify::as.ggplot()
hm_4 <- hm_net(cellchat=cco_4, slot="count", title="Interaction count IFNAR_fl_CpG_D1") %>% ggplotify::as.ggplot()
hm_5 <- hm_net(cellchat=cco_5, slot="count", title="Interaction count IFNAR_fl_CpG_D3") %>% ggplotify::as.ggplot()
hm_6 <- hm_net(cellchat=cco_6, slot="count", title="Interaction count IFNAR_fl_LysM_cre_Baseline_D0") %>% ggplotify::as.ggplot()
hm_7 <- hm_net(cellchat=cco_7, slot="count", title="Interaction count IFNAR_fl_LysM_cre_CpG_D1") %>% ggplotify::as.ggplot()
hm_8 <- hm_net(cellchat=cco_8, slot="count", title="Interaction count IFNAR_fl_LysM_cre_CpG_D3") %>% ggplotify::as.ggplot()

hm_1 + hm_2 + hm_3 + hm_4 + hm_5 + hm_6 + hm_7 + hm_8 + patchwork::plot_layout(nrow=2)

# Compare pathways between sample_group groups

In [ ]:
ery_cells <- c("MEP", "Proerythroblast", "Erythroblast")
myeloid_cells <- c("cMo", "intMo", "ncMo", "RPM")

In [ ]:
options(repr.plot.width=25, repr.plot.height=10)

rnp_1 <- rankNet(cco_diff_1, mode="comparison", stacked=TRUE, sources.use=myeloid_cells, targets.use=ery_cells) + scale_fill_manual(values=color$sample_group) + theme_global_set(1) + theme(axis.text.y=element_text(size=16, color="black"), legend.title=element_blank()) 
rnp_2 <- rankNet(cco_diff_2, mode="comparison", stacked=TRUE, sources.use=myeloid_cells, targets.use=ery_cells) + scale_fill_manual(values=color$sample_group) + theme_global_set(1) + theme(axis.text.y=element_text(size=16, color="black"), legend.title=element_blank()) 
rnp_3 <- rankNet(cco_diff_3, mode="comparison", stacked=TRUE, sources.use=myeloid_cells, targets.use=ery_cells) + scale_fill_manual(values=color$sample_group) + theme_global_set(1) + theme(axis.text.y=element_text(size=16, color="black"), legend.title=element_blank()) 
rnp_4 <- rankNet(cco_diff_4, mode="comparison", stacked=TRUE, sources.use=myeloid_cells, targets.use=ery_cells) + scale_fill_manual(values=color$sample_group) + theme_global_set(1) + theme(axis.text.y=element_text(size=16, color="black"), legend.title=element_blank()) 
rnp_5 <- rankNet(cco_diff_5, mode="comparison", stacked=TRUE, sources.use=myeloid_cells, targets.use=ery_cells) + scale_fill_manual(values=color$sample_group) + theme_global_set(1) + theme(axis.text.y=element_text(size=16, color="black"), legend.title=element_blank()) 

ggpubr::ggarrange(plotlist=list(rnp_1, rnp_2, rnp_3, rnp_4, rnp_5), ncol=5, common.legend=TRUE, legend="none") 